# from week 1

# Loading the vector database

In [1]:
import os
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings

In [10]:
try:
    PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
except:
    print("This should not be running on a notebook.")
    PROJECT_ROOT = os.path.abspath('.')
    
DATA_PATH = os.path.join(PROJECT_ROOT, "database_s3", "queplan_insurance")
DB_PATH = os.path.join(PROJECT_ROOT, "database_chromadb", "db")

This should not be running on a notebook.


In [11]:
vectorstore = Chroma(
    persist_directory=DB_PATH,
    embedding_function=GPT4AllEmbeddings()
)

# First retriever

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
docs = retriever.aget_relevant_documents("Que contrato me recomiendas")

In [15]:
await docs

[Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
 Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contratante del seguro y el Asegurado son personas distintas, corresponde al Tomador o\nContratante el cumplimiento de las obligaciones del contrato, salvo aquellas que por su naturaleza deben ser\ncumplidas por el Asegurado. Las obligaciones del Tomador o Contratante podrán ser cumplidas por el\nAsegurado.', metadata={'page': 12, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\

# First Retrieval Chain

In [17]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_community.llms.openai import OpenAI

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=OpenAI(),
    return_source_documents=True,
    verbose=True
)

In [20]:
def process_llm_response(response):
    print(response['result'])
    print('\n\nSources:')
    [print(source.metadata['source']) for source in response['source_documents']]
    print("\n\n")

In [22]:
msg = 'Que contrato me recomiendas'
response = qa_chain.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [23]:
response

{'query': 'Que contrato me recomiendas',
 'result': ' No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.',
 'source_documents': [Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
  Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contrata

In [24]:
process_llm_response(response)

 No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.


Sources:
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200071.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320180100.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320150503.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200214.pdf





In [25]:
msg = 'Necesito una receta de mayonesa'
response = qa_chain.ainvoke(msg)

In [26]:
result = await response
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Necesito una receta de mayonesa',
 'result': ' No conozco una receta de mayonesa.',
 'source_documents': [Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 1, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320130223.pdf'}),
  Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 16, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320190074.pdf'}),
  Document(page_content='cuyo reembolso se solicita, salvo que se indique otro plazo en las Condiciones Particulares de la póliza. Con\ntodo, en caso de no renovación de la póliza el plazo de aviso de siniestro no podrá ser superior a trein

# Inspeccion de los parametros del Retriever

In [27]:
qa_chain.retriever.search_type

'similarity'

In [ ]:
qa_chain.retriever.search_type = "semantic"

In [28]:
qa_chain.retriever.vectorstore

In [29]:
# System Prompt and Human Prompt:

print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Second Retrieval Chain

In [31]:
from langchain_openai import ChatOpenAI


turbo_llm = ChatOpenAI(
    temperature=0.1777,
    model_name='gpt-3.5-turbo'
)

qa_chain_turbo = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=turbo_llm,
    return_source_documents=True,
    verbose=True
)

In [32]:
msg = 'Requiero recomendaciones para un ARTICULO: "cobertura"'
llm_response = qa_chain_turbo.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
process_llm_response(llm_response)

Para redactar un artículo sobre "cobertura" en el contexto de seguros, podrías considerar los siguientes puntos:

1. **Definición de Cobertura**: Explica qué se entiende por cobertura en el ámbito de los seguros. Puedes mencionar que se refiere a las garantías y protecciones que ofrece una póliza de seguro en caso de determinados eventos.

2. **Tipos de Cobertura**: Describe los diferentes tipos de cobertura que suelen ofrecer las pólizas de seguros, como cobertura médica, cobertura de responsabilidad civil, cobertura de daños materiales, entre otros.

3. **Importancia de la Cobertura**: Destaca la importancia de contar con una cobertura adecuada en un seguro, ya que brinda tranquilidad y protección financiera en situaciones imprevistas.

4. **Periodo de Cobertura**: Menciona que la cobertura de un seguro suele tener un periodo determinado, que puede variar según el tipo de póliza y las condiciones particulares del contrato.

5. **Exclusiones y Limitaciones**: Es importante mencionar q

In [36]:
# System Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [37]:
# Human Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


# from week 2: 29/02/2024

In [1]:
import pickle, os

print(os.getcwd())

with open('pdf_dict.pkl', 'rb') as f:
    data = pickle.load(f)

c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend


In [9]:
data.keys()

dict_keys(['POL120190177.pdf', 'POL320130223.pdf', 'POL320150503.pdf', 'POL320180100.pdf', 'POL320190074.pdf', 'POL320200071.pdf', 'POL320200214.pdf', 'POL320210063.pdf', 'POL320210210.pdf'])

In [10]:
data["POL120190177.pdf"].keys()

dict_keys(['text', 'chunks', 'metadatas', 'retriever'])

In [11]:
type(data["POL120190177.pdf"]["chunks"])
len(data["POL120190177.pdf"]["chunks"])

110

In [12]:
type(data["POL120190177.pdf"]["text"])

str

In [16]:
type(data["POL120190177.pdf"]["retriever"])
"invoke"
dir(data["POL120190177.pdf"]["retriever"])[-50:-24]

['config_specs',
 'configurable_alternatives',
 'configurable_fields',
 'construct',
 'copy',
 'dict',
 'docs',
 'from_documents',
 'from_orm',
 'from_texts',
 'get_graph',
 'get_input_schema',
 'get_lc_namespace',
 'get_name',
 'get_output_schema',
 'get_prompts',
 'get_relevant_documents',
 'input_schema',
 'invoke',
 'is_lc_serializable',
 'json',
 'k',
 'lc_attributes',
 'lc_id',
 'lc_secrets',
 'map']

In [4]:
retriever = data["POL120190177.pdf"]["retriever"]

In [5]:
rta = retriever.invoke("Articulos que mencionen siniestros")
rta

[Document(page_content='aseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que\nocurran con posterioridad a esa fecha.\nARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA\nINDIVIDUAL\nSi durante la vigencia del seguro, un Asegurado Titular deseare incorporar a nuevos asegurados\ndependientes, deberá solicitarlo a la Compañía de Seguros. Para los efectos de incorporarse como\nasegurados, los nuevos dependientes deberán someterse a la evaluación y calificación de su estado de', metadata={'source': '92-lt'}),
 Document(page_content='(15) contado desde la notificación que, conforme al Artículo 21° de estas Condiciones Generales, dirija la\nCompañía Aseguradora al Asegurado y dará derecho al asegurador para exigir que se le pague la prima\ndevengada hasta la fecha de terminación y los gastos de formalización del contrato.\nProducida la terminación, la responsabilidad del asegurador por los siniestros posterior

In [22]:
print(rta[0].page_content)

aseguradora sobre los riesgos que asume y ésta no tendrá obligación alguna respecto de los siniestros que
ocurran con posterioridad a esa fecha.
ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA
INDIVIDUAL
Si durante la vigencia del seguro, un Asegurado Titular deseare incorporar a nuevos asegurados
dependientes, deberá solicitarlo a la Compañía de Seguros. Para los efectos de incorporarse como
asegurados, los nuevos dependientes deberán someterse a la evaluación y calificación de su estado de


In [23]:
print(rta[1].page_content)

(15) contado desde la notificación que, conforme al Artículo 21° de estas Condiciones Generales, dirija la
Compañía Aseguradora al Asegurado y dará derecho al asegurador para exigir que se le pague la prima
devengada hasta la fecha de terminación y los gastos de formalización del contrato.
Producida la terminación, la responsabilidad del asegurador por los siniestros posteriores cesará de pleno
derecho, sin necesidad de declaración judicial alguna.
ARTÍCULO 9º: DENUNCIA DE SINIESTRO 
En caso que el Asegurado, a causa de un Evento, incurra en un gasto susceptible de ser reembolsado o


In [24]:
print(rta[2].page_content)

aportadas, bonificadas o reembolsadas al Asegurado por instituciones o entidades a las que se refiere el
párrafo anterior y que otorguen reintegros, aportes, bonificaciones o reembolsos médicos y prestaciones
económicas al Asegurado.
Queda expresamente establecido que las coberturas de esta Póliza no se duplicarán con los beneficios o
coberturas de cualquier otro seguro, sistema o institución que otorgue beneficios médicos, y al cual
pertenezca el asegurado, de manera que esta Póliza no cubrirá los gastos médicos que deben pagar o


In [25]:
"metadata"
dir(rta[2])[-25:]

['_init_private_attributes',
 '_iter',
 '_lc_kwargs',
 'construct',
 'copy',
 'dict',
 'from_orm',
 'get_lc_namespace',
 'is_lc_serializable',
 'json',
 'lc_attributes',
 'lc_id',
 'lc_secrets',
 'metadata',
 'page_content',
 'parse_file',
 'parse_obj',
 'parse_raw',
 'schema',
 'schema_json',
 'to_json',
 'to_json_not_implemented',
 'type',
 'update_forward_refs',
 'validate']

In [6]:
rta[2].metadata

{'source': '79-lt'}

In [7]:
"docs"
dir(retriever)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_ref

In [11]:
"docs"
retriever.docs
retriever.docs[0].metadata

{'source': '0-lt'}

In [13]:
# Obtaining the relevant document arround the recomendation
# Inspecting recomendation number 3 with index 2

recomendation = rta[2].metadata['source']
for i in range(len(retriever.docs)):
    if recomendation == retriever.docs[i].metadata['source']:
        relevant_docs = [
            retriever.docs[i-2].page_content,
            retriever.docs[i-1].page_content,
            retriever.docs[i].page_content,
            retriever.docs[i+1].page_content,
            retriever.docs[i+2].page_content
        ]
        break



In [14]:
print(relevant_docs[0])
print(relevant_docs[1])
print(relevant_docs[2])
print(relevant_docs[3])
print(relevant_docs[4])

ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES
El Asegurado, conforme al Artículo 9°, letra b), deberá declarar en su denuncia de siniestro sobre el sistema
previsional o de salud al que se encuentre afiliado o adherido, Isapres, Fonasa, cajas de previsión,
departamentos o servicios de bienestar, el Seguro Obligatorio de Accidentes Personales establecido en la
Ley Nº 18.490, las Cajas de Compensación de Asignación Familiar, otros Seguros de Salud contratados en
favor del Asegurado por otras instituciones, públicas o privadas, u otros seguros contratados en forma
individual por él, que contemplen algún aporte, indemnización, reembolso o bonificación, a favor del
Asegurado, efectuado o por efectuar, respecto de los gastos susceptibles de ser reembolsados o pagados al
Prestador.
Para los efectos de esta Póliza, el procedimiento de cálculo de los Gastos Reembolsables considerará
únicamente los Gastos Efectivamente Incurridos, esto es, la diferencia entre el monto total de los Gastos
Méd